<a href="https://colab.research.google.com/github/AnnaZhuravleva/avtobreja/blob/master/Sentiment_for_project_prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install conllu
!pip install tqdm

In [0]:
import numpy as np
import pandas as pd
import os

from conllu import parse
from tqdm import tqdm

### Подготовка данных
Каждый текст - строчка из токенов лемма_тег. Оценки отдельно выведены для сервиса, отдельно для еды. Мы будем строить бинарную классификацию, поэтому будем считать оценки выше 5 положительными, а 5 и ниже — отрицательными.

In [0]:
for instance in list(tqdm._instances): 
    tqdm._decr_instances(instance)

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [0]:
path = "/content/drive/My Drive/studies/HSE/prog/nlp/parsed_train"

In [0]:
!wget https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv

--2019-12-20 15:03:08--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36571759 (35M) [text/plain]
Saving to: ‘SentiRuEval_rest_train.csv’

SentiRuEval_rest_tr 100%[===================>]  34.88M   124MB/s    in 0.3s    

2019-12-20 15:03:11 (124 MB/s) - ‘SentiRuEval_rest_train.csv’ saved [36571759/36571759]



In [0]:
scored = pd.read_csv("SentiRuEval_rest_train.csv", index_col=0)
scored["food"] = scored["food"].apply(int)
scored["service"] = scored["service"].apply(int)
scored.head()

,food,service,text
0,8,8,И пускай на меня не обижается наш прославленны...
1,9,10,"- Здравствуйте. Виа Д’Арженто! - Добрый вечер,..."
2,9,1,"Советую вам уволить Вашего метродотеля Елену, ..."
3,8,9,отличный средне вековый интеръер. Приятное обс...
4,10,8,Ужинали в ресторане Баден-Баден 6 марта . Импо...


In [0]:
num_texts = len(scored.index)
num_texts

19034

In [0]:
preprocessed_texts = []
for i in tqdm(range(num_texts)):
    with open(os.path.join(path, f'{i}.conllu'), "r", encoding='utf-8') as f:
      conllu_text = parse(f.read())    
      lemma_pos_text = []
      for sentence in conllu_text:
          for word in sentence:
              lemma_pos_text.append('_'.join([word["lemma"].lower().strip('-'), word["upostag"]]))
      preprocessed_texts.append(' '.join(lemma_pos_text))

In [0]:
len(preprocessed_texts)

In [0]:
scored["lemma_POS"] = preprocessed_texts
scored.head()

In [0]:
scored.tail()

In [0]:
scored.to_csv("/content/drive/My Drive/studies/HSE/prog/nlp/SentiRuEval_rest_train_lemma_POS.csv")

In [0]:
scored = pd.read_csv("/content/drive/My Drive/studies/HSE/prog/nlp/SentiRuEval_rest_train_lemma_POS.csv", index_col=0)
scored.tail()

,food,service,text,lemma_POS
19029,2,3,"С тех пор, как побывала последний раз, мнение ...","с_ADP тот_DET пора_NOUN ,_PUNCT как_SCONJ быва..."
19030,1,1,Обнаруженный в салате таракан очень поразил!!!...,обнаружить_VERB в_ADP салат_NOUN таракан_NOUN ...
19031,6,5,Забавное заведение. Симпатичный светлый интерь...,забавный_ADJ заведение_NOUN ._PUNCT симпатичны...
19032,8,9,Поздравляем администрацию ресторана Навруз с Н...,поздравлять_VERB администрация_NOUN ресторан_N...
19033,8,8,Очень нравиться интерьер. Отличный гриль.Карта...,очень_ADV нравиться_VERB интерьер_NOUN ._PUNCT...


In [0]:
texts = list(scored["lemma_POS"].apply(lambda s: s.split()))